# Lab 3 - Phân loại thư rác với Naive Bayes

## Thông tin nhóm

## 1. Import thư viện và tải dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter, defaultdict
import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Đọc dữ liệu
print("Đang tải dữ liệu...")
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')

print(f"Kích thước tập train: {train_data.shape}")
print(f"Kích thước tập validation: {val_data.shape}")
print("\n5 dòng đầu tiên của tập train:")
print(train_data.head())

## 2. Tiền xử lý dữ liệu 

### 2.1 Kiểm tra và làm sạch dữ liệu

In [ ]:
# Kiểm tra dữ liệu null
print("Kiểm tra dữ liệu null:")
print(train_data.isnull().sum())
print("\nKiểm tra dữ liệu null trong val set:")
print(val_data.isnull().sum())

# Điền giá trị rỗng cho Message nếu có
train_data['Message'] = train_data['Message'].fillna('')
val_data['Message'] = val_data['Message'].fillna('')

# Kiểm tra dữ liệu trùng lặp
print(f"\nSố dòng trùng lặp trong train: {train_data.duplicated().sum()}")
print(f"Số dòng trùng lặp trong val: {val_data.duplicated().sum()}")

# Loại bỏ dòng trùng lặp nếu có
train_data = train_data.drop_duplicates().reset_index(drop=True)
val_data = val_data.drop_duplicates().reset_index(drop=True)

# Kiểm tra phân phối nhãn
print("\nPhân phối nhãn trong tập train:")
print(train_data['Spam/Ham'].value_counts())
print("\nPhân phối nhãn trong tập val:")
print(val_data['Spam/Ham'].value_counts())

### 2.2 Tiền xử lý văn bản

In [ ]:
def preprocess_text(text):
    """
    Tiền xử lý văn bản:
    - Chuyển về chữ thường
    - Loại bỏ ký tự đặc biệt, giữ lại chữ và số
    - Loại bỏ khoảng trắng thừa
    """
    if pd.isna(text):
        return ""
    
    # Chuyển về chữ thường
    text = text.lower()
    
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ, số và khoảng trắng
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    
    # Loại bỏ khoảng trắng thừa
    text = ' '.join(text.split())
    
    return text

# Áp dụng tiền xử lý cho dữ liệu
print("Đang tiền xử lý văn bản...")
train_data['processed_subject'] = train_data['Subject'].apply(preprocess_text)
train_data['processed_message'] = train_data['Message'].apply(preprocess_text)
train_data['combined_text'] = train_data['processed_subject'] + ' ' + train_data['processed_message']

val_data['processed_subject'] = val_data['Subject'].apply(preprocess_text)
val_data['processed_message'] = val_data['Message'].apply(preprocess_text)
val_data['combined_text'] = val_data['processed_subject'] + ' ' + val_data['processed_message']

print("Ví dụ văn bản sau khi tiền xử lý:")
print(train_data[['Subject', 'processed_subject']].head(3))

## 3. Xây dựng mô hình Naive Bayes từ đầu 

### 3.1 Lý thuyết Naive Bayes

Naive Bayes Classifier cho phân loại văn bản:

1. Theo Bayes' theorem:
   P(class|document) = P(document|class) * P(class) / P(document)

2. Với giả định Naive Bayes (độc lập có điều kiện):
   P(document|class) = P(word1|class) * P(word2|class) * ... * P(wordn|class)

3. Sử dụng log probability để tránh underflow:
   log P(class|document) = log P(class) + Σ log P(word|class)

4. Laplace smoothing để xử lý từ chưa xuất hiện:
   P(word|class) = (count(word, class) + α) / (count(all words in class) + α * |V|)
   với α = 1 (Laplace smoothing), |V| là kích thước từ vựng

### 3.2 Cài đặt Naive Bayes Classifier


In [ ]:
class NaiveBayesClassifier:
    def __init__(self, alpha=1.0):
        """
        Khởi tạo Naive Bayes Classifier
        
        Parameters:
        - alpha: Tham số smoothing (mặc định = 1.0 cho Laplace smoothing)
        """
        self.alpha = alpha
        self.class_priors = {}
        self.word_probs = {}
        self.classes = []
        self.vocab = set()
        
    def _tokenize(self, text):
        """Tách văn bản thành các từ"""
        return text.split()
    
    def fit(self, X_train, y_train):
        """
        Huấn luyện mô hình Naive Bayes
        
        Parameters:
        - X_train: Danh sách văn bản đã tiền xử lý
        - y_train: Nhãn tương ứng
        """
        # Lấy danh sách các lớp
        self.classes = list(set(y_train))
        n_docs = len(y_train)
        
        # Tính prior probability cho mỗi lớp
        print("Tính prior probability...")
        for c in self.classes:
            self.class_priors[c] = sum(1 for y in y_train if y == c) / n_docs
            print(f"P({c}) = {self.class_priors[c]:.4f}")
        
        # Đếm từ cho mỗi lớp
        word_counts = {c: Counter() for c in self.classes}
        doc_counts = {c: 0 for c in self.classes}
        
        print("\nĐang đếm từ...")
        for text, label in zip(X_train, y_train):
            words = self._tokenize(text)
            doc_counts[label] += 1
            for word in words:
                word_counts[label][word] += 1
                self.vocab.add(word)
        
        vocab_size = len(self.vocab)
        print(f"Kích thước từ vựng: {vocab_size}")
        
        # Tính likelihood với Laplace smoothing
        print("\nTính likelihood với Laplace smoothing...")
        self.word_probs = {c: {} for c in self.classes}
        
        for c in self.classes:
            total_words = sum(word_counts[c].values())
            print(f"\nLớp {c}: {total_words} từ")
            
            # Tính P(word|class) cho mỗi từ trong vocabulary
            for word in self.vocab:
                count = word_counts[c].get(word, 0)
                # Laplace smoothing
                self.word_probs[c][word] = (count + self.alpha) / (total_words + self.alpha * vocab_size)
            
            # Tính xác suất cho từ chưa từng xuất hiện (unknown words)
            self.word_probs[c]['<UNK>'] = self.alpha / (total_words + self.alpha * vocab_size)
        
        print("\nHuấn luyện hoàn tất!")
        return self
    
    def _predict_single(self, text):
        """
        Dự đoán nhãn cho một văn bản
        
        Returns: (predicted_class, log_probabilities)
        """
        words = self._tokenize(text)
        
        # Tính log probability cho mỗi lớp
        log_probs = {}
        
        for c in self.classes:
            # Bắt đầu với log prior
            log_prob = math.log(self.class_priors[c])
            
            # Cộng log likelihood của mỗi từ
            for word in words:
                if word in self.word_probs[c]:
                    log_prob += math.log(self.word_probs[c][word])
                else:
                    # Từ chưa xuất hiện, sử dụng xác suất UNK
                    log_prob += math.log(self.word_probs[c]['<UNK>'])
            
            log_probs[c] = log_prob
        
        # Chọn lớp có log probability cao nhất
        predicted_class = max(log_probs, key=log_probs.get)
        
        return predicted_class, log_probs
    
    def predict(self, X_test):
        """Dự đoán nhãn cho tập văn bản"""
        predictions = []
        for text in X_test:
            pred, _ = self._predict_single(text)
            predictions.append(pred)
        return predictions
    
    def predict_proba(self, X_test):
        """
        Dự đoán xác suất cho mỗi lớp
        
        Returns: List of dictionaries với xác suất cho mỗi lớp
        """
        proba_list = []
        
        for text in X_test:
            _, log_probs = self._predict_single(text)
            
            # Chuyển log probabilities về probabilities
            # Sử dụng log-sum-exp trick để tránh overflow
            max_log_prob = max(log_probs.values())
            exp_probs = {c: math.exp(log_prob - max_log_prob) 
                        for c, log_prob in log_probs.items()}
            
            # Normalize
            total = sum(exp_probs.values())
            probs = {c: exp_prob / total for c, exp_prob in exp_probs.items()}
            
            proba_list.append(probs)
        
        return proba_list

### 3.3 Huấn luyện mô hình

In [ ]:
# Khởi tạo và huấn luyện mô hình
nb_classifier = NaiveBayesClassifier(alpha=1.0)

# Chuẩn bị dữ liệu
X_train = train_data['combined_text'].values
y_train = train_data['Spam/Ham'].values

print("Bắt đầu huấn luyện mô hình...")
nb_classifier.fit(X_train, y_train)

### 3.4 Đánh giá mô hình

In [ ]:
def evaluate_model(y_true, y_pred):
    """
    Đánh giá mô hình với các metrics:
    - Accuracy
    - Precision, Recall, F1-score cho mỗi lớp
    - Confusion Matrix
    """
    from collections import Counter
    
    # Accuracy
    accuracy = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred)) / len(y_true)
    
    # Confusion Matrix và metrics cho mỗi lớp
    classes = list(set(y_true))
    metrics = {}
    
    for c in classes:
        # True Positive, False Positive, False Negative, True Negative
        tp = sum((y_t == c) and (y_p == c) for y_t, y_p in zip(y_true, y_pred))
        fp = sum((y_t != c) and (y_p == c) for y_t, y_p in zip(y_true, y_pred))
        fn = sum((y_t == c) and (y_p != c) for y_t, y_p in zip(y_true, y_pred))
        tn = sum((y_t != c) and (y_p != c) for y_t, y_p in zip(y_true, y_pred))
        
        # Precision, Recall, F1
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics[c] = {
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'tp': tp, 'fp': fp, 'fn': fn, 'tn': tn
        }
    
    return accuracy, metrics

# Đánh giá trên tập train
print("Đánh giá trên tập train:")
train_predictions = nb_classifier.predict(X_train)
train_accuracy, train_metrics = evaluate_model(y_train, train_predictions)

print(f"Accuracy: {train_accuracy:.4f}")
for c in train_metrics:
    m = train_metrics[c]
    print(f"\nLớp '{c}':")
    print(f"  Precision: {m['precision']:.4f}")
    print(f"  Recall: {m['recall']:.4f}")
    print(f"  F1-score: {m['f1']:.4f}")

# Đánh giá trên tập validation
print("\n" + "="*50)
print("Đánh giá trên tập validation:")
X_val = val_data['combined_text'].values
y_val = val_data['Spam/Ham'].values

val_predictions = nb_classifier.predict(X_val)
val_accuracy, val_metrics = evaluate_model(y_val, val_predictions)

print(f"Accuracy: {val_accuracy:.4f}")
for c in val_metrics:
    m = val_metrics[c]
    print(f"\nLớp '{c}':")
    print(f"  Precision: {m['precision']:.4f}")
    print(f"  Recall: {m['recall']:.4f}")
    print(f"  F1-score: {m['f1']:.4f}")

# Confusion Matrix
print("\nConfusion Matrix (Val set):")
print("Predicted ->")
print("Actual ↓    spam    ham")
for true_class in ['spam', 'ham']:
    print(f"{true_class:8s}", end="")
    for pred_class in ['spam', 'ham']:
        count = sum((y_t == true_class) and (y_p == pred_class) 
                   for y_t, y_p in zip(y_val, val_predictions))
        print(f"{count:8d}", end="")
    print()

## 4. Thử nghiệm thực tế

### 4.1 Chức năng 1: Dự đoán email người dùng nhập

In [ ]:
def predict_user_email():
    """
    Cho phép người dùng nhập email và dự đoán spam/ham
    """
    print("=== DỰ ĐOÁN EMAIL ===")
    subject = input("Nhập tiêu đề email: ")
    message = input("Nhập nội dung email: ")
    
    # Tiền xử lý
    processed_subject = preprocess_text(subject)
    processed_message = preprocess_text(message)
    combined = processed_subject + ' ' + processed_message
    
    # Dự đoán
    prediction, log_probs = nb_classifier._predict_single(combined)
    
    # Tính xác suất
    max_log_prob = max(log_probs.values())
    exp_probs = {c: math.exp(log_prob - max_log_prob) 
                for c, log_prob in log_probs.items()}
    total = sum(exp_probs.values())
    probs = {c: exp_prob / total for c, exp_prob in exp_probs.items()}
    
    print(f"\nKết quả dự đoán: {prediction.upper()}")
    print(f"Xác suất spam: {probs['spam']:.4f}")
    print(f"Xác suất ham: {probs['ham']:.4f}")
    
    return prediction, probs

In [ ]:
# Test chức năng
# predict_user_email()

### 4.2 Chức năng 2: Đánh giá file CSV


In [ ]:
def evaluate_csv_file(filename):
    """
    Đọc và đánh giá mô hình trên file CSV
    File CSV cần có cấu trúc giống val.csv
    """
    print(f"=== ĐÁNH GIÁ FILE: {filename} ===")
    
    try:
        # Đọc file
        data = pd.read_csv(filename)
        print(f"Đã đọc {len(data)} dòng từ file")
        
        # Kiểm tra cấu trúc
        required_cols = ['Subject', 'Message', 'Spam/Ham']
        if not all(col in data.columns for col in required_cols):
            print("Lỗi: File không có đủ các cột cần thiết!")
            print(f"Cần có: {required_cols}")
            print(f"File có: {list(data.columns)}")
            return
        
        # Tiền xử lý
        data['Message'] = data['Message'].fillna('')
        data['processed_subject'] = data['Subject'].apply(preprocess_text)
        data['processed_message'] = data['Message'].apply(preprocess_text)
        data['combined_text'] = data['processed_subject'] + ' ' + data['processed_message']
        
        # Dự đoán
        X_test = data['combined_text'].values
        y_test = data['Spam/Ham'].values
        predictions = nb_classifier.predict(X_test)
        
        # Đánh giá
        accuracy, metrics = evaluate_model(y_test, predictions)
        
        print(f"\nKết quả đánh giá:")
        print(f"Accuracy: {accuracy:.4f}")
        
        for c in metrics:
            m = metrics[c]
            print(f"\nLớp '{c}':")
            print(f"  Precision: {m['precision']:.4f}")
            print(f"  Recall: {m['recall']:.4f}")
            print(f"  F1-score: {m['f1']:.4f}")
        
        # Chi tiết một số dự đoán
        print("\n5 dự đoán đầu tiên:")
        for i in range(min(5, len(data))):
            print(f"{i+1}. Thực tế: {y_test[i]}, Dự đoán: {predictions[i]}")
            
    except Exception as e:
        print(f"Lỗi khi đọc file: {e}")

In [ ]:
# Test với file val.csv
# evaluate_csv_file('val.csv')

## 5. Phân tích và cải thiện


### 5.1 Phân tích các từ quan trọng


In [ ]:
def get_top_words_per_class(n=20):
    """
    Lấy n từ có xác suất cao nhất cho mỗi lớp
    """
    print(f"=== TOP {n} TỪ QUAN TRỌNG CHO MỖI LỚP ===")
    
    for c in nb_classifier.classes:
        # Lấy log odds ratio: log(P(word|class)) - log(P(word|not_class))
        word_scores = {}
        
        for word in nb_classifier.vocab:
            if word in nb_classifier.word_probs[c]:
                # Log probability của từ trong lớp hiện tại
                log_prob_c = math.log(nb_classifier.word_probs[c][word])
                
                # Log probability trung bình của từ trong các lớp khác
                other_classes = [cls for cls in nb_classifier.classes if cls != c]
                avg_log_prob_other = sum(math.log(nb_classifier.word_probs[cls].get(word, 
                                                  nb_classifier.word_probs[cls]['<UNK>'])) 
                                        for cls in other_classes) / len(other_classes)
                
                # Score = difference
                word_scores[word] = log_prob_c - avg_log_prob_other
        
        # Sắp xếp và lấy top n
        top_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:n]
        
        print(f"\nLớp '{c}':")
        for i, (word, score) in enumerate(top_words, 1):
            print(f"  {i:2d}. {word:20s} (score: {score:.4f})")

In [ ]:
get_top_words_per_class(20)

### 5.2 Thử nghiệm với các tham số smoothing khác nhau

In [ ]:
print("=== THỬ NGHIỆM VỚI CÁC GIÁ TRỊ ALPHA KHÁC NHAU ===")

alphas = [0.1, 0.5, 1.0, 2.0, 5.0]
results = []

for alpha in alphas:
    print(f"\nAlpha = {alpha}")
    
    # Train model với alpha mới
    nb_temp = NaiveBayesClassifier(alpha=alpha)
    nb_temp.fit(X_train, y_train)
    
    # Đánh giá
    val_pred = nb_temp.predict(X_val)
    accuracy, _ = evaluate_model(y_val, val_pred)
    
    results.append({'alpha': alpha, 'accuracy': accuracy})
    print(f"Validation Accuracy: {accuracy:.4f}")

# Tìm alpha tốt nhất
best_result = max(results, key=lambda x: x['accuracy'])
print(f"\nAlpha tốt nhất: {best_result['alpha']} với accuracy: {best_result['accuracy']:.4f}")

## 6. Ví dụ test thực tế

In [ ]:
test_emails = [
    {
        'subject': 'Chúc mừng! Bạn đã trúng thưởng 1 triệu đô la',
        'message': 'Click vào đây ngay để nhận giải thưởng của bạn. Ưu đãi có hạn!'
    },
    {
        'subject': 'Họp nhóm dự án',
        'message': 'Chào các bạn, ngày mai chúng ta sẽ họp lúc 2h chiều để thảo luận về tiến độ dự án.'
    },
    {
        'subject': 'GIẢM GIÁ 90% - MUA NGAY',
        'message': 'Khuyến mãi đặc biệt chỉ hôm nay! Mua ngay kẻo lỡ!!!'
    }
]

In [ ]:
print("=== TEST VỚI CÁC EMAIL MẪU ===")
for i, email in enumerate(test_emails, 1):
    print(f"\nEmail {i}:")
    print(f"Tiêu đề: {email['subject']}")
    print(f"Nội dung: {email['message'][:50]}...")
    
    # Tiền xử lý và dự đoán
    combined = preprocess_text(email['subject']) + ' ' + preprocess_text(email['message'])
    prediction, _ = nb_classifier._predict_single(combined)
    
    print(f"Dự đoán: {prediction.upper()}")

## 7. Kết luận


In [ ]:
print("\n=== TÓM TẮT KẾT QUẢ ===")
print(f"1. Kích thước dữ liệu:")
print(f"   - Train: {len(train_data)} emails")
print(f"   - Validation: {len(val_data)} emails")
print(f"   - Từ vựng: {len(nb_classifier.vocab)} từ")

print(f"\n2. Hiệu suất mô hình:")
print(f"   - Train accuracy: {train_accuracy:.4f}")
print(f"   - Validation accuracy: {val_accuracy:.4f}")

print(f"\n3. Tham số mô hình:")
print(f"   - Smoothing (alpha): {nb_classifier.alpha}")
print(f"   - Phương pháp: Naive Bayes với Laplace smoothing")

print("\n4. Ưu điểm của mô hình:")
print("   - Đơn giản, dễ hiểu và cài đặt")
print("   - Huấn luyện nhanh")
print("   - Hoạt động tốt với dữ liệu văn bản")
print("   - Xử lý tốt với từ chưa xuất hiện nhờ smoothing")

print("\n5. Hạn chế và cải thiện:")
print("   - Giả định độc lập có điều kiện có thể không phù hợp")
print("   - Có thể cải thiện bằng cách:")
print("     + Sử dụng n-grams thay vì unigrams")
print("     + Feature engineering tốt hơn")
print("     + Kết hợp với các đặc trưng khác (độ dài email, số lượng ký tự đặc biệt, ...)")